Introduction

Import libraries and packages

Dataset Information

Separate 'Title' field and preprocess it

Removing unwanted charaters and words

Tokenization and word_index (vocabulary)

Convert titles into sequences and Make n_gram model

Make all titles with same length and padding them

Preprare features (X) and labels (Y)

Architechture of Bidirectional LSTM neural network

Train Bi-LSTM neural network

Plotting accuracy and loss graph

Predict new title (Testing)

Introduction

Next Word Prediction (also called Language Modeling) is the task of predicting what word comes next. It is one of the fundamental tasks of NLP.


In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding,Bidirectional
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import files
upload=files.upload()


Saving medium_data.csv to medium_data.csv


In [3]:
df=pd.read_csv('medium_data.csv')

In [4]:
df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [5]:
df.shape

(6508, 10)

Dataset information

Import Medium-articles-dataset:

This dataset contains information about randomly chosen medium articles published in 2019 from these 7 publications:

Towards Data Science

UX Collective

The Startup

The Writing Cooperative

Data Driven Investor

Better Humans

Better Marketing





In [6]:
df.shape

(6508, 10)

In [7]:
df['title']

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

**Removing unwanted characters and words in titles**

Looking at titles, we can see there are some of unwanted characters and words in it which can not be useful for us to predict infact it might decrease our model accuracy so we have to remove it.

In [8]:
df['title'] = df['title'].apply(lambda x: x.replace(u'\xa0',u' '))
df['title'] = df['title'].apply(lambda x: x.replace('\u200a',' '))

Tokenzation

Tokenzaion is the process in which we provide an unique id to all the words and make a word index or we can say vocabulary.

In [9]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['title'])
total_word=len(tokenizer.index_word)+1
print(total_word)

8237


In [10]:
tokenizer.word_index

{'to': 1,
 'the': 2,
 'strong': 3,
 'a': 4,
 'of': 5,
 'how': 6,
 'and': 7,
 'in': 8,
 'your': 9,
 'markup': 10,
 'for': 11,
 'you': 12,
 'with': 13,
 'is': 14,
 '—': 15,
 'data': 16,
 'why': 17,
 'class': 18,
 'h3': 19,
 'i': 20,
 'what': 21,
 'on': 22,
 'learning': 23,
 'from': 24,
 'an': 25,
 'be': 26,
 'my': 27,
 'writing': 28,
 'are': 29,
 'it': 30,
 'can': 31,
 'using': 32,
 'design': 33,
 'machine': 34,
 'ux': 35,
 'about': 36,
 'do': 37,
 'not': 38,
 'python': 39,
 'ai': 40,
 'life': 41,
 'that': 42,
 'when': 43,
 'should': 44,
 'we': 45,
 '5': 46,
 'science': 47,
 'make': 48,
 'time': 49,
 'need': 50,
 'as': 51,
 '3': 52,
 'more': 53,
 'at': 54,
 'business': 55,
 'or': 56,
 'part': 57,
 'have': 58,
 'work': 59,
 'new': 60,
 'don’t': 61,
 'up': 62,
 'by': 63,
 'write': 64,
 'get': 65,
 'use': 66,
 'guide': 67,
 'will': 68,
 'marketing': 69,
 '1': 70,
 'ways': 71,
 'deep': 72,
 'best': 73,
 'analysis': 74,
 'first': 75,
 '2019': 76,
 'product': 77,
 'better': 78,
 'things': 79,


Titles text into sequences and make n_gram model

suppose, we have sentence like "i am rahul" and this will convert into a sequence with their respective tokens {'i': 1,'am': 2,'rahul': 3}. Thus, output will be [ '1' ,'2' ,'3' ]

Likewise, our all titles will be converted into sequences.

Then, we will make a n_gram model for good prediction.



In [11]:
my_input_sequence =[]
for line in df['title']:
  line.split('\n')
  token_list=tokenizer.texts_to_sequences([line])[0]
  print(token_list)
  for i in range(1,len(token_list)):
    my_n_gram=token_list[:i+1]
    print(my_n_gram)
    my_input_sequence.append(my_n_gram)
print(my_input_sequence)


Streaming output truncated to the last 5000 lines.
[2373, 178]
[2373, 178, 8]
[2373, 178, 8, 3616]
[2373, 178, 8, 3616, 7830]
[2373, 178, 8, 3616, 7830, 1808]
[118, 1, 598]
[118, 1]
[118, 1, 598]
[46, 79, 42, 1503, 512, 485, 303, 170, 371]
[46, 79]
[46, 79, 42]
[46, 79, 42, 1503]
[46, 79, 42, 1503, 512]
[46, 79, 42, 1503, 512, 485]
[46, 79, 42, 1503, 512, 485, 303]
[46, 79, 42, 1503, 512, 485, 303, 170]
[46, 79, 42, 1503, 512, 485, 303, 170, 371]
[46, 130, 20, 103, 24, 1040, 115, 27, 7831, 3387, 262]
[46, 130]
[46, 130, 20]
[46, 130, 20, 103]
[46, 130, 20, 103, 24]
[46, 130, 20, 103, 24, 1040]
[46, 130, 20, 103, 24, 1040, 115]
[46, 130, 20, 103, 24, 1040, 115, 27]
[46, 130, 20, 103, 24, 1040, 115, 27, 7831]
[46, 130, 20, 103, 24, 1040, 115, 27, 7831, 3387]
[46, 130, 20, 103, 24, 1040, 115, 27, 7831, 3387, 262]
[2328, 235, 616, 46, 131, 1, 265, 9, 1160]
[2328, 235]
[2328, 235, 616]
[2328, 235, 616, 46]
[2328, 235, 616, 46, 131]
[2328, 235, 616, 46, 131, 1]
[2328, 235, 616, 46, 131, 1, 2

In [12]:
# pad sequences

max_sequence_len=max([len(seq) for seq in my_input_sequence])
input_sequence=np.array(pad_sequences(my_input_sequence,maxlen=max_sequence_len,padding='pre'))
print(input_sequence[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   4 675]


In [13]:
x=input_sequence[:,:-1]
y=input_sequence[:,-1]

In [14]:
x[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

In [15]:
y

array([675,  67,   1, ...,  55, 729, 549], dtype=int32)

In [16]:
# y convert one hot vector

y=np.array(tf.keras.utils.to_categorical(y,num_classes=total_word))

In [17]:
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [18]:
from keras.api._v2.keras import activations
model=Sequential()
model.add(Embedding(total_word,100,input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_word,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 100)           823700    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8237)              1243787   
                                                                 
Total params: 2218087 (8.46 MB)
Trainable params: 2218087 (8.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x,y,epochs=100,verbose=1)

Epoch 1/100
1515/1515 [==============================] - 43s 23ms/step - loss: 7.0562 - accuracy: 0.0746
Epoch 2/100
1515/1515 [==============================] - 18s 12ms/step - loss: 6.2568 - accuracy: 0.1346
Epoch 3/100
1515/1515 [==============================] - 18s 12ms/step - loss: 5.7710 - accuracy: 0.1609
Epoch 4/100
1515/1515 [==============================] - 17s 11ms/step - loss: 5.3409 - accuracy: 0.1837
Epoch 5/100
1515/1515 [==============================] - 18s 12ms/step - loss: 4.9250 - accuracy: 0.2058
Epoch 6/100
1515/1515 [==============================] - 18s 12ms/step - loss: 4.5208 - accuracy: 0.2303
Epoch 7/100
1515/1515 [==============================] - 17s 11ms/step - loss: 4.1350 - accuracy: 0.2625
Epoch 8/100
1515/1515 [==============================] - 17s 11ms/step - loss: 3.7652 - accuracy: 0.3044
Epoch 9/100
1515/1515 [==============================] - 17s 11ms/step - loss: 3.4136 - accuracy: 0.3516
Epoch 10/100
1515/1515 [==============================]

In [28]:
input_text='India'
pridict_next_word=10
for _ in range(pridict_next_word):
  token_list=tokenizer.texts_to_sequences([input_text])[0]
  print(token_list)
  token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
  predicted=np.argmax(model.predict(token_list),axis=1)
  output_word=" "
  for word,index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
      break
  input_text+=" "+output_word
print(input_text)

[1288]
1/1 [==============================] - 0s 19ms/step
[1288, 1]
1/1 [==============================] - 0s 22ms/step
[1288, 1, 7771]
1/1 [==============================] - 0s 19ms/step
[1288, 1, 7771, 415]
1/1 [==============================] - 0s 21ms/step
[1288, 1, 7771, 415, 7772]
1/1 [==============================] - 0s 25ms/step
[1288, 1, 7771, 415, 7772, 13]
1/1 [==============================] - 0s 23ms/step
[1288, 1, 7771, 415, 7772, 13, 101]
1/1 [==============================] - 0s 20ms/step
[1288, 1, 7771, 415, 7772, 13, 101, 330]
1/1 [==============================] - 0s 18ms/step
[1288, 1, 7771, 415, 7772, 13, 101, 330, 8]
1/1 [==============================] - 0s 22ms/step
[1288, 1, 7771, 415, 7772, 13, 101, 330, 8, 7773]
1/1 [==============================] - 0s 22ms/step
India to punish bitcoin holders with 10 years in prison matching
